# Modified Node Voltage Analysis

[Source](https://lpsa.swarthmore.edu/Systems/Electrical/mna/MNA3.html)



In [1]:
import numpy as np
import matplotlib.pyplot as plt #used for graphing
import numpy.linalg as la

Read circuit and store circuit elements in a list, with each element as a dictionary.

In [98]:
# Voltage Divider
# spice = """Vs1 0 1 1.5
# R1 1 2 100
# R2 2 0 50"""

# Current Divider
# spice = """Is1 0 1 3
# R1 0 1 50
# R2 0 1 100"""

# Example 2
# spice = """Vs1 1 2 32
# Vs2 0 3 20
# R1 0 1 2
# R2 0 2 8
# R3 2 3 4"""

# Example 3
spice = """Is1 0 1 0.01
R1 0 1 50
R2 1 2 100
R3 0 2 200
Vs1 2 1 5"""

circuit = spice.split('\n')
print("circuit: ", circuit)

nodes = {
    "neg": [],
    "pos": []
}
Vsources = []
Rs = []
Isources = []

for line in circuit:
    element, neg, pos, val = line.split(' ')
    neg = int(neg)
    pos = int(pos)
    val = float(val)
    name = element[1:]
    if neg not in nodes['neg']:
        nodes['neg'].append(neg)
    if pos not in nodes['pos']:
        nodes['pos'].append(pos)
        
    elementDict = {
        "name": name,
        "neg": neg,
        "pos": pos,
        "val": val
    }
    
    if element[0] == 'V':
        Vsources.append(elementDict)
    elif element[0] == "I":
        Isources.append(elementDict)
    elif element[0] == "R":
        Rs.append(elementDict)

n = np.max(nodes['pos'])
m = len(Vsources)

print("Number of nodes, n = ", n)
print("Number of V sources, m = ", m)
print("V sources: ",Vsources)
print("I sources: ",Isources)
print("Resistors: ",Rs)
print("Neg nodes:", nodes['neg'])
print("Pos nodes:", nodes['pos'])

circuit:  ['Is1 0 1 0.01', 'R1 0 1 50', 'R2 1 2 100', 'R3 0 2 200', 'Vs1 2 1 5']
Number of nodes, n =  2
Number of V sources, m =  1
V sources:  [{'name': 's1', 'neg': 2, 'pos': 1, 'val': 5.0}]
I sources:  [{'name': 's1', 'neg': 0, 'pos': 1, 'val': 0.01}]
Resistors:  [{'name': '1', 'neg': 0, 'pos': 1, 'val': 50.0}, {'name': '2', 'neg': 1, 'pos': 2, 'val': 100.0}, {'name': '3', 'neg': 0, 'pos': 2, 'val': 200.0}]
Neg nodes: [0, 1, 2]
Pos nodes: [1, 2]


In [99]:
A = np.zeros((n+m, n+m)) #initialize 2D array of size n+m x n+m
z = np.zeros(n+m) #initialize 1D array is size n+m x 1

#initialize arrays that make up A
G = np.zeros((n, n))
B = np.zeros((n, m))
C = np.zeros((m, n))
D = np.zeros((m, m))


In [100]:
#create G matrix
#diagonal
for i in range(1,n+1):
    cond = 0
    for res in Rs:
        if res['neg']==i:
            R = res['val']
            cond = cond + 1/R
        elif res['pos']==i:
            R = res['val']
            cond = cond + 1/R
    G[i-1,i-1] = cond
#off diagonal
for i in range(1,n+1):
    for j in range(1,n+1):
        for res in Rs:
            if (res['neg']==i and res['pos']==j) or (res['neg']==j and res['pos']==i):
                cond = 1/res['val']
                G[i-1,j-1] = -cond

print(G)

[[ 0.03  -0.01 ]
 [-0.01   0.015]]


In [101]:
#create B matrix
for s in range(1,m+1):
    nnode = Vsources[s-1]['neg']
    pnode = Vsources[s-1]['pos']
    if(nnode!=0):
        B[nnode-1,s-1] = -1
    if(pnode!=0):
        B[pnode-1,s-1] = 1

print(B)

[[ 1.]
 [-1.]]


In [102]:
#create C matrix
for s in range(1,m+1):
    nnode = Vsources[s-1]['neg']
    pnode = Vsources[s-1]['pos']
    if(nnode!=0):
        C[s-1,nnode-1] = -1
    if(pnode!=0):
        C[s-1,pnode-1] = 1

print(C)

[[ 1. -1.]]


In [103]:
#create z matrix from i and e matrices
iarr = np.zeros(n)
earr = np.zeros(m)

#create i matrix
for node in range(1,n+1):
    for s in Isources:
        if s['pos'] == node:
            iarr[node-1] = iarr[node-1] + s['val']

#create e matrix
for v in range(1,m+1):
    earr[v-1] = Vsources[v-1]['val']
    
#create z matrix
z[:n] = iarr
z[n:] = earr

print(z)

[0.01 0.   5.  ]


In [104]:
A[:n,:n] = G[:,:]
A[:n,n:] = B[:,:]
A[n:,:n] = C[:,:]
A[n:,n:] = D[:,:]

print(A)

[[ 0.03  -0.01   1.   ]
 [-0.01   0.015 -1.   ]
 [ 1.    -1.     0.   ]]


Solve for node voltages and currents through voltage sources.

In [105]:
#solve for x in Ax = z
Ainv = la.inv(A)
x = Ainv@z
x

array([ 1.4  , -3.6  , -0.068])